## Data Preprocessing
: 분석에 이용할 연도별 팀투수 및 팀타자 데이터의 전처리

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### 1. 파생변수 생성

투수/타자 평가에 사용되는 여러 지표들을 계산 후 사용. 
지표 계산에 사용되는 연도별 상수들은 조사 후, 기존에 계산된 값들을 찾아 사용함.

- 투수: K/9, BB/9, K/BB, OAVG, OOBP, OSLG, OOPS, WHIP, BABIP, DICE, ERA, RA9, FIP, kFIP, HR/9, H/9 
- 타자: AVG, SLG, IsoP, OBP, OPS, RC, XR, wOBA, BABIP

In [ ]:
# 투수/ 타자 파생변수 생성 위한 연도별 상수들을 데이터프레임 형태로, idx_pit 및 idx_hit이라는 이름으로 저장
idx_pit= pd.DataFrame({'GYEAR': [2020, 2019, 2018, 2017, 2016], 'cFIP': [3.597, 3.403, 3.833, 3.735, 3.792]})
idx_pit

In [ ]:
idx_hit = pd.DataFrame({'GYEAR': [2020, 2019, 2018, 2017, 2016], 'wBB': [0.733, 0.721, 0.726, 0.728, 0.747],
                       'wHBP': [0.761, 0.752, 0.753, 0.756, 0.774], 'w1B': [0.939, 0.945, 0.920, 0.927, 0.944],
                       'w2B': [1.282, 1.317, 1.242, 1.259, 1.273], 'w3B': [1.592, 1.653, 1.533, 1.558, 1.569],
                       'wHR': [1.950, 2.074, 1.857, 1.901, 1.898]})
idx_hit

In [ ]:
#투수 파생변수 생성 함수 정의
def add_variables_pit(data, year):
    data = pd.read_csv(data)
    data['H1']= data['HIT']-data['H2']-data['H3']-data['HR']
    
    data.loc[data['G_ID'].str.contains('%d' % year), 'GYEAR'] = year
    data = pd.merge(data, idx_pit, on='GYEAR')
    
    data = data.sort_values(by=['T_ID', 'GDAY_DS'])
    data= data.drop('GYEAR', axis=1 )

    data = data.sort_index()

    data['K/9'] = 9*data['KK']/(data['INN2']/3)
    data['BB/9'] = 9*data['BB']/(data['INN2']/3)
    data['K/BB'] = data['KK']/data['BB']
    data['OAVG'] = data['HIT']/data['AB']
    data['OOBP'] = (data['HIT'] + data['BB'] + data['HP'])/(data['AB'] + data['BB'] +data['HP'] + data['SF'])
    data['OSLG'] = (data['H1'] + 2*data['H2'] + 3*data['H3'] + 4*data['HR'])/data['AB']
    
    data = data.replace([np.nan, np.inf, -np.inf], 0)
    data['OOPS'] = data['OOBP'] + data['OSLG']
    data['WHIP'] = (data['HIT'] + data['BB'])/(data['INN2']/3)
    data['BABIP'] = (data['HIT'] - data['HR'])/(data['AB'] - data['KK'] - data['HR'] + data['SF'])
    data['DICE']= 3.00 + (13*data['HR'] + 3*(data['BB'] + data['HP']) - 2*data['KK'])/(data['INN2']/3)
    data['ERA'] = 9*data['ER']/(data['INN2']/3)
    data['RA9'] = 9 * data['R']/(data['INN2']/3)
    data['FIP'] = data['cFIP'] + (13*data['HR'] + 3*(data['BB'] + data['HP']) - 2*data['KK'])/(data['INN2']/3)
    data['kFIP'] = data['cFIP'] + (14* data['HR'] + 3*(data['BB'] + data['HP']) - data['KK'])/(data['INN2']/3)
    data['HR/9'] = 9 * data['HR']/(data['INN2']/3)
    data['H/9'] = 9 * data['HIT']/(data['INN2']/3)

    data = data.replace([np.nan, np.inf, -np.inf], 0)
    data = data.drop(['cFIP'],axis=1)
    data=data[['G_ID', 'GDAY_DS', 'T_ID', 'VS_T_ID', 'HEADER_NO', 'TB_SC', 'CG_CK',
       'WLS', 'HOLD', 'INN2', 'BF', 'PA', 'AB', 'HIT','H1', 'H2', 'H3', 'HR', 'SB',
       'CS', 'SH', 'SF', 'BB', 'IB', 'HP', 'KK', 'GD', 'WP', 'BK', 'ERR', 'R',
       'ER', 'P_WHIP_RT', 'P2_WHIP_RT', 'CB_WHIP_RT', 'K/9', 'BB/9',
       'K/BB', 'OAVG', 'OOBP', 'OSLG', 'OOPS', 'WHIP', 'BABIP', 'DICE', 'ERA',
       'RA9', 'FIP', 'kFIP', 'HR/9', 'H/9']]
    data.to_csv('team_pitcher_%d.csv' % year, index=False)

In [ ]:
#투수 파생변수 계산 함수 적용, 저장
add_variables_pit('2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2016.csv', 2016)
add_variables_pit('2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2017.csv', 2017)
add_variables_pit('2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2018.csv', 2018)
add_variables_pit('2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2019.csv', 2019)
add_variables_pit('2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2020.csv', 2020)

In [ ]:
#타자 파생변수 생성함수 정의
def add_variables_hit(data, year):
    data = pd.read_csv(data)
    data['H1']= data['HIT']- data['H2'] - data['H3'] - data['HR']
    
    data.loc[data['G_ID'].str.contains('%d' % year), 'GYEAR'] = year
    data = pd.merge(data, idx_hit, on='GYEAR')
    
    data = data.sort_values(by=['T_ID', 'G_ID'])
    data= data.drop('GYEAR', axis=1 )
    
    data['AVG'] = data['HIT']/ data['AB']
    data['SLG'] = (data['H1']+ 2 * data['H2']+ 3 * data['H3'] + 4 *data['HR']) / data['AB']
    data = data.replace([np.nan, np.inf, -np.inf], 0)
    data['IsoP']= data['SLG']- data['AVG']               
    data['OBP']= (data['HIT'] + data['BB'] + data['HP'])/(data['AB']+data['BB']+data['HP']+data['SF'])
    data = data.replace([np.nan, np.inf, -np.inf], 0)
    data['OPS'] = data['OBP']+ data['SLG']
    
    data['RC']= ((data['HIT']+ data['BB']+ data['HP']-data['CS']-data['GD']) *  ((data['H1']+ 2 * data['H2'] + 3 * data['H3'] + 4 * data['HR']) + 0.52 * (data['SB']+data['SH']+data['SF']) + 0.26 * (data['BB']+ data['HP'] -data['IB'])))/(data['AB']+data['BB']+data['HP']+data['SH']+data['SF'])
    data['XR']= 0.5*data['H1'] + 0.72*data['H2']+ 1.04 *data['H3'] + 1.44 *data['HR'] + 0.34* (data['HP']+data['BB']-data['IB'])+ 0.25* data['IB'] + 0.18 * data['SB'] - 0.32*data['CS'] - 0.09 * (data['AB'] -data['HIT']-data['KK']) - 0.098 * data['KK'] -0.37 *data['GD'] + 0.37* data['SF']+ 0.04* data['SH']
    data['wOBA']= (data['wBB'] * (data['BB']-data['IB']) + data['wHBP']*data['HP'] + data['w1B']*data['H1']+data['w2B']* data['H2'] + data['w3B'] * data['H3'] +data['wHR']*data['HR'])/(data['AB']+data['BB']-data['IB']+ data['SF']+ data['HP'])
    data['wOBA']= (data['wBB'] * (data['BB']-data['IB']) + data['wHBP']*data['HP'] + data['w1B']*data['H1']+data['w2B']* data['H2'] + data['w3B'] * data['H3'] +data['wHR']*data['HR'])/(data['PA']-data['IB'])
    data['BABIP']= (data['HIT'] - data['HR']) / (data['AB']-data['KK']-data['HR']+data['SF'])
    
    data = data.replace([np.nan, np.inf, -np.inf], 0)
    data = data.drop(['wBB','wHBP','w1B','w2B','w3B','wHR'],axis=1)
    data = data[['G_ID', 'GDAY_DS', 'T_ID', 'VS_T_ID', 'HEADER_NO', 'TB_SC', 'PA', 'AB',
       'RBI', 'RUN', 'HIT','H1', 'H2', 'H3', 'HR', 'SB', 'CS', 'SH', 'SF', 'BB',
       'IB', 'HP', 'KK', 'GD', 'ERR', 'LOB', 'P_HRA_RT', 'P_AB_CN', 'P_HIT_CN',
       'AVG', 'SLG', 'IsoP', 'OBP', 'OPS', 'RC', 'XR', 'wOBA', 'BABIP']]
    
    data.to_csv('team_batter_%d.csv' % year, index=False)

In [ ]:
#타자 파생변수 계산 함수 적용, 저장
add_variables_hit('2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2016.csv', 2016)
add_variables_hit('2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2017.csv', 2017)
add_variables_hit('2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2018.csv', 2018)
add_variables_hit('2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2019.csv', 2019)
add_variables_hit('2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2020.csv', 2020)

In [ ]:
#투수 데이터 병합; 연도별 팀투수 데이터를 tpit이라는 이름으로 하나의 데이터로 병합.
pit16 = pd.read_csv('team_pitcher_2016.csv')
pit17 = pd.read_csv('team_pitcher_2017.csv')
pit18 = pd.read_csv('team_pitcher_2018.csv')
pit19 = pd.read_csv('team_pitcher_2019.csv')
pit20 = pd.read_csv('team_pitcher_2020.csv')

pit = pd.concat([pit16, pit17, pit18, pit19, pit20],axis=0)


#타자 데이터 병합; 연도별 팀타자 데이터를 thit이라는 이름으로 하나의 데이터로 병합.
hit16 = pd.read_csv('team_batter_2016.csv')
hit17 = pd.read_csv('team_batter_2017.csv')
hit18 = pd.read_csv('team_batter_2018.csv')
hit19 = pd.read_csv('team_batter_2019.csv')
hit20 = pd.read_csv('team_batter_2020.csv')

hit = pd.concat([hit16, hit17, hit18, hit19, hit20],axis=0)

### 2. Check NAs
: 확인결과 타자/투수 데이터 모두 NA값 없음을 확인

In [ ]:
pit.isnull().sum()

In [ ]:
hit.isnull().sum()

### 3. Feature Selection: remove correlated variables

**Variable Selection Criteria**
1. 상관계수 0.9 이상 변수
2. 상관관계가 0.9 이상인, 기존변수 vs. 파생변수 $\rightarrow$ 기존변수를 남기고, 파생변수를 제거
3. 상관관계가 0.9 이상인,기존변수 vs. 기존변수 or 파생변수 vs. 파생변수 $\rightarrow$ 주관적 판단에 의해 변수선택

***1) 투수 데이터(Team Pitchers data)***

In [ ]:
col = ['G_ID', 'GDAY_DS', 'T_ID', 'VS_T_ID', 'HEADER_NO', 'TB_SC', 'CG_CK','WLS', 'HOLD', 'INN2', 'BF']
data_cor = pit.drop(col,axis=1)
fig, ax = plt.subplots(figsize=(30,30))
ax = sns.heatmap(data_cor.corr(), annot=True, fmt='.1f',linewidth=1)

**High-Correlated Variables**

- PA와 AB$\rightarrow$ 보류, 일단 둘다 사용
- kFIP, FIP, DICE: 계산 과정에서 계수에 약간에 차이를 둔 FIP 계열의 지표들. $\rightarrow$ kFIP만 사용
- OAVG, OOBP, OSLG, OOPS, WHIP $\rightarrow$ OOPS, WHIP만 사용

**$\rightarrow$ Feature Selection 결과**: kFIP, OOPS, WHIP, BABIP, K/BB와 기존 변수들.

In [ ]:
pit = pit.drop(['K/9','BB/9','OAVG','OOBP','OSLG','DICE','ERA','RA9','FIP','HR/9','H/9'],axis=1)

***2) 타자 데이터(Team Hitters data)***

In [ ]:
col = ['G_ID', 'GDAY_DS', 'T_ID', 'VS_T_ID', 'HEADER_NO', 'TB_SC']
data_cor = hit.drop(col, axis=1)
fig, ax = plt.subplots(figsize=(30,30))
ax = sns.heatmap(data_cor.corr(), annot=True, fmt='.1f',linewidth=1)

**High-Correlated Variables**

- PA(타자수) vs. AB(타수) $\rightarrow$ 보류, 둘다 보존
- AVG(타율) vs. SLG(장타율), OBP(출루율), OPS(장타율+출루율), wOBA(가중출루율), BABIP $\rightarrow$ AVG(타율)만 보존
- IsoP(순장타율) 보존
- RC(득점기여도) vs. XR(득점공헌도) $\rightarrow$ XR 선택

**$\rightarrow$ Feature Selection 결과**: 파생변수 중에서는 XR, OPS, IsoP, BABIP만 선택, 기존변수는 그대로 사용


- 의미가 없다고 판단되는 변수: LOB(잔루) $\rightarrow$ 제거

In [ ]:
hit = hit.drop(['AVG','SLG','OBP','RC','wOBA', 'LOB'],axis=1)

### 4. Scaling & Dummy화

***1) 투수 데이터 (Team Pitchers data)***
- AB(타수), HIT(안타), H1, H2, H3, HR, SH(희타), SF(희플), BB(볼넷), HP(사구), KK(삼진) 등은 타석에 포함됨  $\rightarrow$ PA(타석)으로 나누어서 경기별로 scaling 시킴
- PA(타석), P_WHIP_RT, K/BB, kFIP, WHIP, ERR(실책), GD(병살타), OOPS,BF(투구수)  $\rightarrow$ MinMaxScaler 사용
- BABIP은 이미 0~1사이의 값을 가지기 때문에 따로 스케일링하지 않음.
- HEADER_NO(더블해더), TB_SC(초/말), WB(폭투), BK(보크)  $\rightarrow$ dummy화

In [ ]:
# 타석으로 나누는 작업
col = ['AB','HIT','H1','H2','H3','HR','SH','SF','BB','IB','HP','KK']
for i in range(len(col)):
    pit.loc[:,col[i]] = pit.loc[:,col[i]]/pit.loc[:,'PA']

In [ ]:
# MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(pit[['PA','P_WHIP_RT','P2_WHIP_RT','CB_WHIP_RT','K/BB','kFIP','WHIP','ERR','GD','OOPS','BF']])
pit.loc[:,['PA','P_WHIP_RT','P2_WHIP_RT','CB_WHIP_RT','K/BB','kFIP','WHIP','ERR','GD','OOPS','BF']] = scaler.transform(pit[['PA','P_WHIP_RT','P2_WHIP_RT','CB_WHIP_RT','K/BB','kFIP','WHIP','ERR','GD','OOPS','BF']])

In [ ]:
print(pit.WP.value_counts()) #WP(폭투)의 경우 2이상의 폭투 여부 (0,1)로 dummy화하는 것이 낫다고 판단.
print(pit.BK.value_counts()) #BK(보크)의 경우 2이상의 폭투 여부 (0,1)로 dummy화하는 것이 낫다고 판단.

In [ ]:
# dummy화
pit['WP'] = pit['WP'].replace([1,2,3,4,5],1)    # 폭투 여부를 0,1로 더미화    
pit['BK'] = pit['BK'].replace([1,2],1) # 보크 여부를 0,1로 더미화
pit=pd.get_dummies(pit, columns=['HEADER_NO','TB_SC'],  drop_first=True)

***2) 타자 데이터 (Team Hitters Data)***
- 타석(PA)에 포함되는 AB,HIT(안타수), H1, H2, H3, HR, SH(희타),SF(희비),BB(볼넷),HP(사구), KK(삼진)$\rightarrow$ 타석으로 나누어 scaling
- PA(타석), XR(추정득점), GD(병살타), ERR(실책), OPS $\rightarrow$ MinMaxScaler사용
- 이미 0~1사이 값을 갖는 IsoP, BABIP, PA_HRA_RT(득점권 타율)은 따로 scaling하지 않음.
- HEADER_NO(더블헤더), TB_SC(초/말) dummy화


In [ ]:
# 타석으로 나누어 Scaling
col = ['AB','HIT','H1','H2','H3','HR','SH','SF','BB','HP','IB','KK','P_AB_CN', 'P_HIT_CN']
for i in range(len(col)):
    hit.loc[:,col[i]] = hit.loc[:,col[i]]/hit.loc[:,'PA']

In [ ]:
# MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(hit[['PA','XR','GD','ERR','OPS']])
hit.loc[:,['PA','XR','GD','ERR','OPS']] = scaler.transform(hit[['PA','XR','GD','ERR','OPS']])

In [ ]:
# HEADER_NO, TB_SC dummy화
hit=pd.get_dummies(hit, columns=['HEADER_NO','TB_SC'],  drop_first=True)

### 5. 기타 처리

In [ ]:
#데이터 정렬
pit=pit.sort_values(by=['G_ID', 'T_ID']).reset_index(drop=True)
hit= hit.sort_values(by=['G_ID', 'T_ID']).reset_index(drop=True)

In [ ]:
#16~20년도 경기 중, 주어진 데이터에서 무승부는 단 66경기뿐.
#무승부 경기는 예측을 위해 제외하고 생각하기.
temp= []
for i in range(len(pit['WLS'])):
    if pit.loc[i,'WLS']!='D' :
        temp.append(i)
pit=pit.iloc[temp,:].reset_index(drop=True)
hit=hit.iloc[temp,:].reset_index(drop=True)

In [ ]:
#GDAY_DS는 날짜형식으로 바꿔주기
pit['GDAY_DS']=pd.to_datetime(pit['GDAY_DS'].astype('str'))
hit['GDAY_DS']=pd.to_datetime(hit['GDAY_DS'].astype('str'))

In [ ]:
d

In [ ]:
pit.columns

In [ ]:
hit.columns

In [ ]:
#투수, 타자 데이터를 통합해 total이라는 이름으로 저장.
data_pit=pit
data_pit.columns=['G_ID', 'GDAY_DS', 'T_ID', 'VS_T_ID', 'CG_CK', 'WLS', 'HOLD', 'INN2',
       'BF', 'PIT_PA', 'PIT_AB', 'PIT_HIT', 'PIT_H1', 'PIT_H2', 'PIT_H3', 'PIT_HR', 'PIT_SB', 'PIT_CS', 'PIT_SH', 'PIT_SF',
       'PIT_BB', 'PIT_IB', 'PIT_HP', 'PIT_KK', 'PIT_GD', 'PIT_WP', 'PIT_BK', 'PIT_ERR', 'PIT_R', 'PIT_ER', 'P_WHIP_RT',
       'P2_WHIP_RT', 'CB_WHIP_RT', 'K/BB', 'OOPS', 'WHIP', 'PIT_BABIP', 'kFIP',
       'HEADER_NO_1', 'HEADER_NO_2', 'TB_SC_T']
data_hit=hit
data_hit.columns=['G_ID', 'GDAY_DS', 'T_ID', 'VS_T_ID', 'HIT_PA', 'HIT_AB', 'HIT_RBI', 'HIT_RUN', 'HIT_HIT',
       'HIT_H1', 'HIT_H2', 'HIT_H3', 'HIT_HR', 'HIT_SB', 'HIT_CS', 'HIT_SH', 'HIT_SF', 'HIT_BB', 'HIT_IB', 'HIT_HP', 'HIT_KK',
       'HIT_GD', 'HIT_ERR', 'P_HRA_RT', 'P_AB_CN', 'P_HIT_CN', 'IsoP', 'OPS', 'XR',
       'HIT_BABIP', 'HEADER_NO_1', 'HEADER_NO_2', 'TB_SC_T']

total=pd.merge(data_pit, data_hit)
total.head()

pit: 투수지표
hit: 타자지표
total: 투수지표/ 타자지표 통합 ver.

In [ ]:
import seaborn as sns

In [ ]:
pit.columns

In [ ]:
sns.distplot(hit['RUN'], kde=False)

## Modelling